# Simple 2 Layer CNN on TensorFlow with ReLU Activation

This CNN is a 2 Layer convnet running on TensorFlow and using ReLu activation, while also importing the dataset from a local folder. The code was a modified version of the one found here: https://raw.githubusercontent.com/aymericdamien/TensorFlow-Examples/master/examples/5_DataManagement/build_an_image_dataset.py

In [1]:
from __future__ import print_function

import tensorflow as tf
import os

# disables warning for my CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
config = tf.ConfigProto()
config.intra_op_parallelism_threads = 44
config.inter_op_parallelism_threads = 44

# Dataset Parameters - CHANGE HERE
MODE = 'folder'
TRAIN_DATASET_PATH = 'C:/Users/chris/Documents/CUNY/DATA698 - Final Masters Thesis/Images/take2/train'
TEST_DATASET_PATH = 'C:/Users/chris/Documents/CUNY/DATA698 - Final Masters Thesis/Images/take2/test'

# Image Parameters
N_CLASSES = 7
IMG_HEIGHT = 224
IMG_WIDTH = 224
CHANNELS = 3 # The 3 color channels, or 1 if grayscale

In [ ]:
# Reading the dataset
def read_images(dataset_path, mode, batch_size):
    imagepaths, labels = list(), list()
    if mode == 'file':
        # Read dataset file
        data = open(dataset_path, 'r').read().splitlines()
        for d in data:
            imagepaths.append(d.split(' ')[0])
            labels.append(int(d.split(' ')[1]))
    elif mode == 'folder':
        # An ID will be affected to each sub-folders by alphabetical order
        label = 0
        # List the directory
        try:  # Python 2
            classes = sorted(os.walk(dataset_path).next()[1])
        except Exception:  # Python 3
            classes = sorted(os.walk(dataset_path).__next__()[1])
        # List each sub-directory (the classes)
        for c in classes:
            c_dir = os.path.join(dataset_path, c)
            try:  # Python 2
                walk = os.walk(c_dir).next()
            except Exception:  # Python 3
                walk = os.walk(c_dir).__next__()
            # Add each image to the training set
            for sample in walk[2]:
                # Only keeps jpeg images
                if sample.endswith('.jpg') or sample.endswith('.jpeg'):
                    imagepaths.append(os.path.join(c_dir, sample))
                    labels.append(label)
            label += 1
    else:
        raise Exception("Unknown mode.")

    # Convert to Tensor
    imagepaths = tf.convert_to_tensor(imagepaths, dtype=tf.string)
    labels = tf.convert_to_tensor(labels, dtype=tf.int32)
    # Build a TF Queue, shuffle data
    image, label = tf.train.slice_input_producer([imagepaths, labels],
                                                 shuffle=True)

    # Read images from disk
    image = tf.read_file(image)
    image = tf.image.decode_jpeg(image, channels=CHANNELS)

    # Resize images to a common size
    image = tf.image.resize_images(image, [IMG_HEIGHT, IMG_WIDTH])

    # Normalize
    image = image * 1.0/127.5 - 1.0

    # Create batches
    X, Y = tf.train.batch([image, label], batch_size=batch_size,
                          capacity=batch_size * 8,
                          num_threads=4)

    return X, Y

In [2]:
# Parameters
learning_rate = 0.001
num_steps = 100
batch_size = 75
display_step = 10

# Network Parameters
dropout = 0.75 # Dropout, probability to keep units

# Build the data inputs
X, Y = read_images(TRAIN_DATASET_PATH, MODE, batch_size)
X2, Y2 = read_images(TEST_DATASET_PATH, MODE, batch_size)

In [3]:
# Create model
def conv_net(x, n_classes, dropout, reuse, is_training):
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):

        # Convolution Layer with 32 filters and a kernel size of 5
        conv1 = tf.layers.conv2d(x, 32, 5, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)

        # Convolution Layer with 32 filters and a kernel size of 5
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)

        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv2)

        # Fully connected layer (in contrib folder for now)
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc1, n_classes)
        # Because 'softmax_cross_entropy_with_logits' already apply softmax,
        # we only apply softmax to testing network
        out = tf.nn.softmax(out) if not is_training else out

    return out

# Because Dropout has different behavior at training and prediction time, we
# need to create 2 distinct computation graphs that share the same weights.

# Create a graph for training
logits_train = conv_net(X, N_CLASSES, dropout, reuse=False, is_training=True)
# Create another graph for testing that reuse the same weights
logits_test = conv_net(X2, N_CLASSES, dropout, reuse=True, is_training=False)

# Define loss and optimizer (with train logits, for dropout to take effect)
loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits=logits_train, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits_test, 1), tf.cast(Y, tf.int64))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Saver object
saver = tf.train.Saver()

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
# Start training
with tf.Session(config=config) as sess:

    # Run the initializer
    sess.run(init)

    # Start the data queue
    tf.train.start_queue_runners()

    # Training cycle
    for step in range(1, num_steps+1):

        if step % display_step == 0:
            # Run optimization and calculate batch loss and accuracy
            _, loss, acc = sess.run([train_op, loss_op, accuracy])
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
        else:
            # Only run the optimization op (backprop)
            sess.run(train_op)

    print("Optimization Finished!")

    # Save your model
    # saver.save(sess, 'my_tf_model')

Step 10, Minibatch Loss= 2.2272, Training Accuracy= 0.080
Step 20, Minibatch Loss= 0.4120, Training Accuracy= 0.173
Step 30, Minibatch Loss= 0.1594, Training Accuracy= 0.147
Step 40, Minibatch Loss= 0.0632, Training Accuracy= 0.067
Step 50, Minibatch Loss= 0.0501, Training Accuracy= 0.093
Step 60, Minibatch Loss= 0.0035, Training Accuracy= 0.133
Step 70, Minibatch Loss= 0.0211, Training Accuracy= 0.147
Step 80, Minibatch Loss= 0.0284, Training Accuracy= 0.147
Step 90, Minibatch Loss= 0.0524, Training Accuracy= 0.093
Step 100, Minibatch Loss= 0.0558, Training Accuracy= 0.107
Optimization Finished!
ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[Node: batch_1/fifo_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_FLOAT, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](batch_1/fifo_queue, sub_1, input_producer_1/Gather_1)]]
ERROR:tensorflow:Exception in QueueRunner: Enqueue operation was cancelled
	 [[Node: input_producer_1/inpu

Exception in thread QueueRunnerThread-batch/fifo_queue-batch/fifo_queue_enqueue:
Traceback (most recent call last):
  File "c:\program files\python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\program files\python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "c:\program files\python36\lib\site-packages\tensorflow\python\training\queue_runner_impl.py", line 252, in _run
    enqueue_callable()
  File "c:\program files\python36\lib\site-packages\tensorflow\python\client\session.py", line 1249, in _single_operation_run
    self._call_tf_sessionrun(None, {}, [], target_list, None)
  File "c:\program files\python36\lib\site-packages\tensorflow\python\client\session.py", line 1420, in _call_tf_sessionrun
    status, run_metadata)
  File "c:\program files\python36\lib\site-packages\tensorflow\python\framework\errors_impl.py", line 516, in __exit__
    c_api.TF_GetCode(self.status.status))
tensorflow.python.framework.

NameError: name 'data' is not defined